# Market Observations 2: The NASDAQ Whale

By: Chengyi (Jeff) Chen

Over the weekend, [The Financial Times ](https://www.ft.com/content/75587aa6-1f1f-4e9d-b334-3ff866753fa2) released a piece revealing that SoftBank Group has been buying up 4 billion dollars worth of shares of Apple, Amazon, Microsoft, Tesla and roughly the same amount of out of the money call options. This alone, implies that SoftBank believes that the current tech stock rally will continue on into the future. This move has been described by The Financial Times, Wall Street Journal, CNBC, and other sources to have a substantial contribution to the tech rally we have seen in the past month. From my understanding, the impact proceeds as follows: 
1. SoftBank purchases 4 billion dollars worth of out of the money (otm) call options
2. The counterparty that sells SoftBank these call options in turn, take on a net delta negative position, so in order to delta hedge, they purchase shares of the underlying stock of the call options to reduce the negative delta position
3. This in turn creates upward pressure for the underlying tech stock prices which turn the otm call options closer to in the money (itm)
4. With an increase in the underlying stock price, and because SoftBank's otm calls have a positive gamma position, delta becomes even more positive
5. Because SoftBank's call options have an increasingly positive delta position, the counterparty has to buy even more of the underlying stock to further hedge it's position closer to delta neutral
6. This creates a ["gamma feedback loop"](https://www.zerohedge.com/markets/classic-feedback-loop-why-everyone-chasing-gamma-crash), repeating events 3 - 5 until the market corrects itself (Still unsure about how this happens - At what point in time will the counterparty decide to sell off the underlying stock / positive delta position investors decide to sell off)

The extreme volatility in tech stocks started last Thursday when the markets were down, [attributed to "profit-taking"](https://www.marketwatch.com/story/dow-looks-set-to-pull-back-thursday-as-it-closes-in-on-record-ahead-of-jobless-claims-report-2020-09-03), while the Nasdaq tumble on Tuesday September 8th was fuelled by [doubts on whether the COVID-19 infections will resurge in the coming winter months and uncertainty about the US Elections and the US-China economic "decoupling"](https://edition.cnn.com/2020/09/08/investing/nasdaq-stock-market-today/index.html). The Nasdaq tumble led to a 10 billion dollar loss in SoftBank's market valuation. It appears to me that there could also be more players like SoftBank that are making dangerous bets on tech stocks through the use of options, without doing their fair share of due diligence on the underlying company - ["(t)he whole strategy is just momentum buying"](https://www.ft.com/content/53aa19bb-fc9a-46fd-aafd-4bc3669ab161). The doubt on how big an impact SoftBank had in the tech rally is also supported by this [Bloomberg article](https://www.bloomberg.com/news/articles/2020-09-07/nasdaq-whale-theory-for-tech-stock-surge-is-stirring-doubts), which argues that retail investors could also have contributed to the high volatility in tech stocks.

Resources:
- [projectoption's YouTube video: How SoftBank's Billion-Dollar Option Bets Helped Fuel the Stock Market Rally](https://www.youtube.com/watch?v=fm2cnP1ruag)

In [95]:
%load_ext nb_black
%load_ext autotime

import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.figsize"] = (12, 8)

import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import plotly.express as px


def get_all_option_quotes(ticker: yf.ticker.Ticker, option_type="calls"):
    """Retrieves all available option data in a single pandas dataframe"""
    assert (option_type == "calls") | (option_type == "puts"), ValueError(
        "'option_type' must be either 'puts' or 'calls'"
    )
    all_option_quotes_df = None
    try:
        available_option_expiry_dates = ticker.options
        for option_expiry_date in available_option_expiry_dates:
            try:
                df = (
                    ticker.option_chain(option_expiry_date).calls
                    if option_type == "calls"
                    else ticker.option_chain(option_expiry_date).puts
                )
                df["option_expiry_date"] = option_expiry_date
                if all_option_quotes_df is None:
                    all_option_quotes_df = df
                else:
                    all_option_quotes_df = pd.concat([all_option_quotes_df, df], axis=0)
            except:
                print(
                    f"Option data for {ticker.ticker} on {option_expiry_date} is unavailable."
                )
        return all_option_quotes_df
    except:
        print(f"Option data for {ticker.ticker} is unavailable.")
        return None

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


<IPython.core.display.Javascript object>

time: 36 ms


In [93]:
import yfinance as yf

tickers = yf.Tickers("tsla fb vxn").tickers

<IPython.core.display.Javascript object>

time: 23.5 ms


In [86]:
tickers.FB.history(period="max", interval="1d")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2012-05-18,42.05,45.00,38.00,38.23,573576400,0,0
2012-05-21,36.53,36.66,33.00,34.03,168192700,0,0
2012-05-22,32.61,33.59,30.94,31.00,101786600,0,0
2012-05-23,31.37,32.50,31.36,32.00,73600000,0,0
2012-05-24,32.95,33.21,31.77,33.03,50237200,0,0
...,...,...,...,...,...,...,...
2020-09-02,298.88,303.60,293.05,302.50,24341400,0,0
2020-09-03,295.99,297.60,283.63,291.12,32294100,0,0
2020-09-04,287.25,289.00,271.14,282.73,30333700,0,0


<IPython.core.display.Javascript object>

time: 205 ms


In [96]:
df = get_all_option_quotes(ticker=tickers.VXN, option_type="calls")

Option data for VXN is unavailable.


<IPython.core.display.Javascript object>

time: 53.2 ms


In [79]:
df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,option_expiry_date
0,FB200911C00130000,2020-09-08 13:56:17,130.0,145.80,0.0,0.0,0.0,0.0,11.0,0.0,0.000010,1.0,REGULAR,USD,2020-09-11
1,FB200911C00135000,2020-09-09 16:34:24,135.0,141.30,0.0,0.0,0.0,0.0,1.0,0.0,0.000010,1.0,REGULAR,USD,2020-09-11
2,FB200911C00140000,2020-09-08 18:44:08,140.0,132.70,0.0,0.0,0.0,0.0,36.0,0.0,0.000010,1.0,REGULAR,USD,2020-09-11
3,FB200911C00145000,2020-09-08 14:28:31,145.0,130.64,0.0,0.0,0.0,0.0,3.0,0.0,0.000010,1.0,REGULAR,USD,2020-09-11
4,FB200911C00150000,2020-09-02 14:20:27,150.0,147.30,0.0,0.0,0.0,0.0,2.0,0.0,0.000010,1.0,REGULAR,USD,2020-09-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51,FB220916C00400000,2020-09-09 18:41:06,400.0,27.31,0.0,0.0,0.0,0.0,6.0,0.0,0.062509,0.0,REGULAR,USD,2022-09-16
52,FB220916C00410000,2020-09-08 18:21:13,410.0,25.50,0.0,0.0,0.0,0.0,8.0,0.0,0.062509,0.0,REGULAR,USD,2022-09-16
53,FB220916C00420000,2020-09-09 13:58:04,420.0,24.79,0.0,0.0,0.0,0.0,2.0,0.0,0.062509,0.0,REGULAR,USD,2022-09-16
54,FB220916C00430000,2020-09-08 16:08:06,430.0,24.18,0.0,0.0,0.0,0.0,2.0,0.0,0.062509,0.0,REGULAR,USD,2022-09-16


<IPython.core.display.Javascript object>

time: 26.4 ms


In [45]:
px.scatter(data_frame=df, x="", y="strike")

<IPython.core.display.Javascript object>